# Commands

In [1]:
# cd twinews-logs ; jupython -o nohup-tfidf-$HOSTNAME.out --venv st-venv ~/Workspace/Python/Datasets/Twinews/twinews/models/tfidf.ipynb

# Init

In [2]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
isNotebook = '__file__' not in locals()

In [4]:
TEST = isNotebook # isNotebook, True, False

In [5]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from nlptools.preprocessing import *
from twinews.utils import *
from twinews.models.ranking import *

In [6]:
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [7]:
logger = Logger(tmpDir('logs') + "/tfidf.log") if isNotebook else Logger("tfidf-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

# Config

In [8]:
config = \
{
    'splitVersion': 2,
    
    'maxUsers': 2 if TEST else None, # Sub-sampling
    'minDF': None if TEST else None, # Remove words that have a document frequency ratio lower than 1 / 500
    'maxDF': None if TEST else None, # Remove top 300 voc elements
    
    'lowercase': False if TEST else False,
    'doLemmatization': False if TEST else False,
    'sublinearTF': True,
    
    'dimensions': None if TEST else 100, # 20, 50, 100, 500, None (means no dimensionality reduction)
    'maxIter': 100, # 10, 30, 100 (Iterations for SVD)
    
    'distance': 'cosine', # 'cosine', 'euclidean', 'kl', 'js'
    'historyRef': 30, # 1, 1.0, 0.6, 0.3, 3, 10, 30
}

In [9]:
modelName = 'tfidf'

In [10]:
if config['dimensions'] is None:
    del config['maxIter']

In [11]:
# Here we check if we already generated ranking for this model with this specific config:
if not isNotebook:
    if rankingExists(modelName, config, logger=logger):
        raise Exception(modelName + " with this config already exist:\n" + b(config, 5))

# Getting data

In [12]:
# Getting users and news
evalData = getEvalData(config['splitVersion'], maxExtraNews=0, maxUsers=config['maxUsers'], logger=logger)
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['meta'], 5), logger)

--> tic: 32.94s | message: Eval data loaded
--> toc total duration: 33.26s | message: Got Twinews evaluation data
{ candidates, extraNews, meta, testNews, testUsers, trainNews, trainUsers }
{ 'created': 2020.03.24-14.28.06, 'endDate': 2018-01-15, 'id': 2, 'ranksLength': 1000, 'splitDate': 2017-12-25, 'startDate': 2017-10-01, 'testMaxNewsPerUser': 97, 'testMeanNewsPerUser': 7.22, 'testMinNewsPerUser': 2, 'testNewsCount': 71781, 'totalNewsAvailable': 570210, 'trainMaxNewsPerUser': 379, 'trainMeanNewsPerUser': 26.48, 'trainMinNewsPerUser': 8, 'trainNewsCount': 237150, 'usersCount': 15905 }


In [13]:
# Here it is important to convert urls to lists because we want the same order to retrieve vectors by index...
# And we shuffle it so we do not stick urls a a user at the begin...
# But we seed the random to always have same order...
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)
newsList = trainNewsList + testNewsList

In [14]:
# Print all:
log(str(len(trainNewsList)) + " urls for trainNewsList", logger=logger)
log(str(len(testNewsList)) + " urls for testNewsList", logger=logger)
log(str(len(newsList)) + " urls for newsList", logger=logger)

28 urls for trainNewsList
1989 urls for testNewsList
2017 urls for newsList


In [15]:
# We get sentences:
sentences = getNewsSentences(newsList, logger=logger)
bp(sentences, logger)

twinews news (version 1.0) initialised.
  0% [                    ]
  9% [=                   ] (37.042s left)
 19% [===                 ] (33.304s left)
 29% [=====               ] (28.537s left)
 39% [=======             ] (24.078s left)
 49% [=========           ] (19.998s left)
 59% [===========         ] (15.809s left)
 69% [=============       ] (11.883s left)
 79% [===============     ] (7.915s left)
 89% [=================   ] (3.999s left)
 99% [=================== ] (0.134s left)
100% [====================] (total duration: 38.75s, mean duration: 0.019s)
[
  [ [ Site, Mobile, ..., long, . ], [ Credit, Shane, ..., streets, . ], ..., [ Please, verify, ..., box, . ], [ You, are, ..., email, . ] ],
  [ [ What, missing, ..., room, . ], [ ", I, ..., late, ! ], ..., [ I, know, ..., hurting, . ], [ I, know, ..., of, . ] ],
  ...,
  [ [ The, great, ..., awards, . ], [ ", I, ..., Instagram, . ], ..., [ Rep., Bonnie, ..., ignored, . ], [ ", In, ..., said, . ] ],
  [ [ Sheriff, :, ..., W

In [16]:
# We flatten sentences:
for i in range(len(sentences)):
    sentences[i] = flattenLists(sentences[i])
docs = sentences
bp(docs, logger)

[ [ Site, Mobile, ..., email, . ], [ What, missing, ..., of, . ], ..., [ The, great, ..., said, . ], [ Sheriff, :, ..., said, . ] ]


In [17]:
# Lower case:
if config['lowercase']:
    for i in pb(list(range(len(docs))), logger=logger, message="Lower casing"):
        for u in range(len(docs[i])):
            docs[i][u] = docs[i][u].lower()
bp(docs, logger)

[ [ Site, Mobile, ..., email, . ], [ What, missing, ..., of, . ], ..., [ The, great, ..., said, . ], [ Sheriff, :, ..., said, . ] ]


In [18]:
# Lemmatization:
if config['doLemmatization']:
    lemmatizer = WordNetLemmatizer()
    pbar = ProgressBar(len(docs), logger=logger, message="Lemmatization")
    for i in range(len(docs)):
        for u in range(len(docs[i])):
            docs[i][u] = lemmatizer.lemmatize(docs[i][u])
        pbar.tic()
bp(docs, logger)

[ [ Site, Mobile, ..., email, . ], [ What, missing, ..., of, . ], ..., [ The, great, ..., said, . ], [ Sheriff, :, ..., said, . ] ]


In [19]:
# Filtering the corpus:
if config['minDF'] is not None or config['maxDF'] is not None:
    docs = filterCorpus(docs, minDF=config['minDF'], maxDF=config['maxDF'],
                        removeEmptyDocs=False, allowEmptyDocs=False, logger=logger)
    for doc in docs: assert len(doc) > 0
    bp(docs, logger)

In [20]:
tt.tic("Data preprocessed")

--> tic: 1m 12.42s | message: Data preprocessed


72.42

# Infering vectors

In [21]:
vectorizer = TfidfVectorizer\
(
    sublinear_tf=config['sublinearTF'],
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    # lowercase=True, # Doesn't work because we erased preprocessor
)
vectors = vectorizer.fit_transform(docs)
assert vectors.shape[0] == len(newsList)

In [22]:
tt.tic("TFIDF vectors generated")

--> tic: 0.96s | message: TFIDF vectors generated


0.96

# Dimensionality reduction

In [23]:
if config['dimensions'] is not None:
    svd = TruncatedSVD(n_components=config['dimensions'], n_iter=config['maxIter'], random_state=42)
    vectors = svd.fit_transform(vectors)

# Making a dict url --> topic vector

In [24]:
assert len(vectors) == len(newsList)
urlsVectors = dict()
for i in range(len(newsList)):
    urlsVectors[newsList[i]] = vectors[i]
bp(urlsVectors, logger)

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

# Ranking

In [ ]:
# Read the doc!
rankings = usersRankingsByHistoryDistance\
(
    trainUsers,
    candidates,
    config['historyRef'],
    urlsVectors,
    distanceMetric=config['distance'],
    logger=logger,
)

In [ ]:
bp(rankings, logger, 4)

In [ ]:
tt.tic("Rankings done")

# Adding rankings to the db

In [ ]:
# Read the doc!
addRanking(modelName, rankings, config, logger=logger)

In [ ]:
tt.tic("Rankings stored")

In [ ]:
totalDuration = tt.toc()

In [ ]:
notif(modelName + '-' + objectToHash(config)[:5] + " done in " + secondsToHumanReadableDuration(totalDuration) + " on " + getHostname())